# Byte Level Language Model Evaluation

In [1]:
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as K
from tqdm import tqdm
import os
import time
import pandas as pd
import numpy as np
import psutil
# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
%matplotlib inline

In [2]:
print(tf.__version__)
print(keras.__version__)

2.2.0
2.3.0-tf


In [3]:
gpu_devices = tf.config.list_physical_devices('GPU')

if gpu_devices:
    print('Using GPU')
    for gpu in gpu_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print('Using CPU')
    tf.config.optimizer.set_jit(True) #activate Accelerated Linear Algebra Processor (XLA) through C API
    print('used: {}% free: {:.2f}GB'.format(psutil.virtual_memory().percent, float(psutil.virtual_memory().free)/1024**3))#@ 

Using CPU
used: 83.2% free: 1.24GB


In [4]:
from functions import *

In [5]:
# fix random seed for reproducibility
K.clear_session()
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

### IDEA FORMULATION

In [7]:
def make_bitseq(s: str) -> str:
    if not s.isascii():
        raise ValueError("ASCII only allowed")
    return " ".join(f"{ord(i):08b}" for i in s)
make_bitseq('Hello')

'01001000 01100101 01101100 01101100 01101111'

In [8]:
def n_possible_values(nbits: int) -> int:
    return 2 ** nbits
print('6 Bits :', n_possible_values(6))
print('7 Bits :', n_possible_values(7))
print('8 Bits :', n_possible_values(8))

6 Bits : 64
7 Bits : 128
8 Bits : 256


In [22]:
def convert(binary_number):
    binary = binary_number
    i = 0
    decimal_number = 0
    while (binary_number != 0):
        c = int(binary_number % 10)
        decimal_number = decimal_number + c * (2 ** i)
        i +=1
        binary_number = binary_number / 10
    print('Binary number: %d' % binary)
    print('Decimal number: %d' % decimal_number)
    return 0
convert(1010)

Binary number: 1010
Decimal number: 10


0

In [10]:
1 * 2**3 + 0 * 2**2 + 0 * 2**1 + 1 * 2**0

9

"Bottom Line:" Aim is to represent characters that require "1 Byte" and of only "7 Bits slots" (Not 8 Bits as we are used too) covering all english language alphabets, numerics and symbols up too decimal point "127". Decimal point 0 is reserved for padding and whenever model sees a 0 it will ignore it.

### GET DATA

In [13]:
from pathlib import Path
NEWS_STORE = Path('../news_db.h5')

In [14]:
with pd.HDFStore(NEWS_STORE) as store:
    keys = list(store.keys())
    news = pd.DataFrame()
    for i in tqdm(keys): 
        news = news.append(store[i])  

news = news.reset_index().set_index('versionCreated')
news.index = news.index.strftime('%Y-%m-%d %H:%M:%S')
news.index = pd.to_datetime(news.index)
news = news.drop(['index','storyId', 'sourceCode'], axis = 1)
news.index.name = 'time'
news.info()

100%|████████████████████████████████████████████████████████████████████████████████| 542/542 [05:32<00:00,  1.63it/s]


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1153304 entries, 2020-09-01 21:20:54 to 2021-01-20 00:04:31
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   text    1153304 non-null  object
 1   ticker  1153304 non-null  object
dtypes: object(2)
memory usage: 26.4+ MB


In [ ]:
txt = ''
# Count Unique Characters
for doc in text:
    for s in doc:
        txt += s
chars = sorted(set(txt))
print(chars)
print(len(chars))

### PARSE STOP-END TOKENS & ENCODE(1-BYTE)

In [13]:
text = clean_text(news, 'text')  #---> <s> headline <\s> and clean
b_text = encode2bytes(text) #----->ordinal encoding
max_sentence_len = max([len(sentence) for sentence in b_text])

### SPLIT INPUT / TARGET

In [14]:
X, y = split_X_y(b_text)
num = np.random.randint(0, len(X))
print('This is an example of the training sequence encoded as bytes:\n')
print(X[num])
print(text[num])
print(y[num])

This is an example of the training sequence encoded as bytes:

[60, 115, 62, 82, 80, 84, 45, 87, 69, 83, 84, 74, 69, 84, 32, 67, 69, 79, 32, 83, 65, 89, 83, 32, 84, 72, 69, 82, 69, 32, 73, 83, 32, 34, 86, 69, 82, 89, 32, 77, 65, 74, 79, 82, 32, 74, 79, 66, 34, 32, 84, 79, 32, 66, 69, 32, 68, 79, 78, 69, 32, 84, 79, 32, 71, 73, 86, 69, 32, 80, 69, 79, 80, 76, 69, 32, 67, 79, 78, 70, 73, 68, 69, 78, 67, 69, 32, 84, 72, 65, 84, 32, 67, 72, 65, 78, 71, 69, 83, 32, 84, 72, 65, 84, 32, 72, 65, 86, 69, 32, 66, 69, 69, 78, 32, 77, 65, 68, 69, 32, 87, 73, 76, 76, 32, 77, 65, 75, 69, 32, 55, 51, 55, 32, 77, 65, 88, 32, 84, 72, 69, 32, 83, 65, 70, 69, 83, 84, 32, 78, 65, 82, 82, 79, 87, 66, 79, 68, 89, 32, 68, 79, 77, 69, 83, 84, 73, 67, 32, 65, 73, 82, 67, 82, 65, 70, 84, 60, 92, 115]
<s>RPT-WESTJET CEO SAYS THERE IS "VERY MAJOR JOB" TO BE DONE TO GIVE PEOPLE CONFIDENCE THAT CHANGES THAT HAVE BEEN MADE WILL MAKE 737 MAX THE SAFEST NARROWBODY DOMESTIC AIRCRAFT<\s>
[115, 62, 82, 80, 84, 45, 87, 69

### PADDING 
Masking is a way to tell sequence-processing layers that certain timesteps in an input are missing, and thus should be skipped when processing the data.

Padding is a special form of masking where the masked steps are at the start or at the beginning of a sequence. Padding comes from the need to encode sequence data into contiguous batches: in order to make all sequences in a batch fit a given standard length, it is necessary to pad or truncate some sequences.

In [11]:
X = pad_sequences(X, maxlen = max_sentence_len, padding = 'post')
y = pad_sequences(y, maxlen = max_sentence_len, padding = 'post')
print(X.shape, y.shape)

(976043, 519) (976043, 519)


### TEST & VALIDATION SETS

In [12]:
length_train = 959651
train_size = length_train * 90//100

validation_seq_data = tf.data.Dataset.from_tensor_slices((X[train_size:length_train + 1],y[train_size:length_train + 1]))
test_seq_data = tf.data.Dataset.from_tensor_slices((X[length_train + 1: ],y[length_train + 1:]))

In [13]:
print('Check VALIDATION set:')
for input_txt, target_txt in  validation_seq_data.take(1):
    print('--------------------------------Headline--------------------------------')
    print(input_txt.numpy())
    print("".join(map(chr, input_txt.numpy())))
#     print(''.join(index2char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print("".join(map(chr, target_txt.numpy())))

Check VALIDATION set:
--------------------------------Headline--------------------------------
[ 60 115  62  84  82  65  67  84  79  82  32  83  85  80  80  76  89  32
  45  32  83  69  69  83  32  50  48  49  57  32  69  65  82  78  73  78
  71  83  32  80  69  82  32  68  73  76  85  84  69  68  32  83  72  65
  82  69  32  36  52  46  54  53  32  45  32  36  52  46  55  53  60  92
 115   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0 

In [14]:
#Mini-Batching/Subsequencing
batch_size = 256


validation_seq_data = validation_seq_data.batch(batch_size, drop_remainder=True)
test_seq_data = test_seq_data.batch(batch_size, drop_remainder=True)
print('Validation Set Shape: ', validation_seq_data, '\nTest Set Shape: ', test_seq_data)

Validation Set Shape:  <BatchDataset shapes: ((256, 519), (256, 519)), types: (tf.int32, tf.int32)> 
Test Set Shape:  <BatchDataset shapes: ((256, 519), (256, 519)), types: (tf.int32, tf.int32)>


In [18]:
# Memory Buffer for data prefetching 
AUTOTUNE = tf.data.experimental.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

validation_seq_data = configure_dataset(validation_seq_data)
test_seq_data = configure_dataset(test_seq_data)

<img src="../Models/Byte_embeds.png">

### LOAD LANGUAGE MODEL
* If you want to compile: Compile then load in that order only.

In [6]:
model = tf.keras.models.load_model('CharLangModel.h5', compile=False)
model.build(tf.TensorShape([256,None]))
#compile below

In [7]:
model_complile(model)

1

In [8]:
trained_weights = './training_checkpoints_CharWeights'
model.load_weights(tf.train.latest_checkpoint(trained_weights))

### Validation Set Evaluation

In [24]:
score = model.evaluate(validation_seq_data, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

374/374 [==============================] - 30055s 80s/step - loss: 0.0095 - sparse_categorical_accuracy: 0.1564
Test loss: 0.009494862519204617
Test accuracy: 0.15639889240264893


### Test Set Evaluation

In [18]:
model_complile(model, sparse_acc = False)

1

In [19]:
trained_weights = './training_checkpoints_CharWeights'
model.load_weights(tf.train.latest_checkpoint(trained_weights))

In [28]:
validation_score = model.evaluate(test_seq_data, verbose=1)
print('Test loss:', validation_score[0])
print('Test accuracy:', validation_score[1])

64/64 [==============================] - 5429s 85s/step - loss: 0.0155 - accuracy: 0.1557
Test loss: 0.015461504459381104
Test accuracy: 0.1557215005159378


* Observation: I wont judge this model based on its accuracy as not only is my data sparse, but also some features are not present in the dataset.

In [442]:
uniqye_characters = set(x for l in b_text for x in l)
len(uniqye_characters)

97

## Inference 

In [9]:
headline = "<s>ZOETIS INC <ZTS.N>: CREDIT SUISSE RAISES PRICE TARGET TO $192 FROM $182 ZOETIS INC <ZTS.N>: BOFA GLOBAL RESEARCH RAISES PRICE OBJECTIVE TO $175 FROM $170 NYSE ORDER IMBALANCE <ZTS.N> 77562.0 SHARES ON SELL SIDE<\s>"
# Encode UTF-8 ordinal level
headline = encode2bytes(headline)
#Split INput text and Output target
X, y_true = headline[:-1], headline[1:]
# Convert to array and squeeze dimension over axis = 0
X = tf.expand_dims(X, 0)
# Predict Ouput
prediction = model.predict(X.numpy())
prediction.shape

(1, 216, 127)

#### Logits (as is)

In [10]:
print("Input:      ", "".join(map(chr, np.squeeze(X))))
print("Prediction: " ,"".join(map(chr,np.argmax(prediction, axis = -1).squeeze())))
print("Actual:     " ,"".join(map(chr,np.squeeze(y_true))))

Input:       <s>ZOETIS INC <ZTS.N>: CREDIT SUISSE RAISES PRICE TARGET TO $192 FROM $182 ZOETIS INC <ZTS.N>: BOFA GLOBAL RESEARCH RAISES PRICE OBJECTIVE TO $175 FROM $170 NYSE ORDER IMBALANCE <ZTS.N> 77562.0 SHARES ON SELL SIDE<\s
Prediction:  s>JOETIS INC <XTS.N>: CREDIT SUISSE RAISES PRICE TARGET TO $192 FROM $182 QOETIS INC <XTS.N>: BOFA GLOBAL RESEARCH RAISES PRICE OB,ECTIVE TO $175 FROM $170 NYSE ORDER IMBALANCE <ZTS.N> J7562.0 SHARES ON SELL SIDE<\s>
Actual:      s>ZOETIS INC <ZTS.N>: CREDIT SUISSE RAISES PRICE TARGET TO $192 FROM $182 ZOETIS INC <ZTS.N>: BOFA GLOBAL RESEARCH RAISES PRICE OBJECTIVE TO $175 FROM $170 NYSE ORDER IMBALANCE <ZTS.N> 77562.0 SHARES ON SELL SIDE<\s>


#### Softmax

In [126]:
prediction = prediction[-1,:,:]
p_i = np.zeros((prediction.shape))
for i in range(0, len(headline[:-1])):
    p = np.exp(prediction[i])/np.sum(np.exp(prediction[i])) #softmax
    p_i[i] = p

In [127]:
print(np.argmax(p_i, axis = 1))
''.join(map(chr,np.argmax(p_i, axis = 1)))

[115  62  74  79  69  84  73  83  32  73  78  67  32  60  88  84  83  46
  78  62  58  32  67  82  69  68  73  84  32  83  85  73  83  83  69  32
  82  65  73  83  69  83  32  80  82  73  67  69  32  84  65  82  71  69
  84  32  84  79  32  36  49  57  50  32  70  82  79  77  32  36  49  56
  50  32  81  79  69  84  73  83  32  73  78  67  32  60  88  84  83  46
  78  62  58  32  66  79  70  65  32  71  76  79  66  65  76  32  82  69
  83  69  65  82  67  72  32  82  65  73  83  69  83  32  80  82  73  67
  69  32  79  66  44  69  67  84  73  86  69  32  84  79  32  36  49  55
  53  32  70  82  79  77  32  36  49  55  48  32  78  89  83  69  32  79
  82  68  69  82  32  73  77  66  65  76  65  78  67  69  32  60  90  84
  83  46  78  62  32  74  55  53  54  50  46  48  32  83  72  65  82  69
  83  32  79  78  32  83  69  76  76  32  83  73  68  69  60  92 115  62]


's>JOETIS INC <XTS.N>: CREDIT SUISSE RAISES PRICE TARGET TO $192 FROM $182 QOETIS INC <XTS.N>: BOFA GLOBAL RESEARCH RAISES PRICE OB,ECTIVE TO $175 FROM $170 NYSE ORDER IMBALANCE <ZTS.N> J7562.0 SHARES ON SELL SIDE<\\s>'

#### Random Sampling

In [128]:
sampled_indices = tf.random.categorical(prediction, num_samples=1) 
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
"".join(map(chr,sampled_indices))
# wont use sampling in my case

's>JOETIY INC <%TS.N>: CREDIT SUISSE RAISES TRICE TARGET TO $172 FROM $582 JOETIS INC <YTS.N>) BOFA GLOBAL RESEARCH RAISES PRICE OBWECTIIE TO $145 FROM $1%0 NYSE ORDER IMBAAANCE FBTS.N> 6U562.0 SHARES ON SELL SIDE<\\\\>'

In [11]:
#Use generate new text function to test on model
generate_text(model, '<s> Today', 10, 1)
#Concluded model is stateless and only learned how to represent and regenerate passed text but not generate new text!

'<s> Today<>>>>>>>>>'

### EMBEDDINGS LAYER

In [95]:
trained_embeddings = model.get_layer('EmbedLayer').get_weights()[0]

In [100]:
trained_embeddings.shape

(127, 256)

In [124]:
input_seq = '<s>AMZN wont be paying any taxes for 2019~<\s'
input_seq = tf.squeeze(encode2bytes(input_seq)).numpy()
input_seq

array([ 60, 115,  62,  65,  77,  90,  78,  32, 119, 111, 110, 116,  32,
        98, 101,  32, 112,  97, 121, 105, 110, 103,  32,  97, 110, 121,
        32, 116,  97, 120, 101, 115,  32, 102, 111, 114,  32,  50,  48,
        49,  57, 126,  60,  92, 115])

In [125]:
# Process of representing each of our features/character/byte/input
lookup_table = tf.nn.embedding_lookup(trained_embeddings, input_seq)
lookup_table

<tf.Tensor: shape=(45, 256), dtype=float32, numpy=
array([[-0.13882878, -0.30200124,  0.08077791, ...,  0.16514868,
         0.18841438,  0.09872594],
       [-0.11984932, -0.17366889,  0.0651598 , ...,  0.12940577,
        -0.2054549 , -0.05256342],
       [ 0.04611618,  0.08065684,  0.1510432 , ...,  0.01680804,
        -0.09226788, -0.03016171],
       ...,
       [-0.13882878, -0.30200124,  0.08077791, ...,  0.16514868,
         0.18841438,  0.09872594],
       [-0.10369939, -0.25992334,  0.02147431, ...,  0.1398279 ,
         0.11564761,  0.15265961],
       [-0.11984932, -0.17366889,  0.0651598 , ...,  0.12940577,
        -0.2054549 , -0.05256342]], dtype=float32)>

In [127]:
np.all(trained_embeddings[60]) == np.all(lookup_table[0])

True

In a nutshell:

For each character/byte the model looks up the embedding, runs the LSTM one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character/Byte. This distribution, for each predicted character/byte, is defined by the logits over the characters(i.e 1-126 Decimal Points bytes(0 is reserved for padding)).

#### Save Embeddings Representations and Visualize in 3D

In [132]:
import io, csv

# save model weights

print(trained_embeddings.shape) # shape: (characters/bytes, embedding_dim) -->(127,256)

# save embeddings.
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
tsv_writer = csv.writer(out_m, delimiter='\t')


for i in range(0,127):
    if i == 0: continue # skip 0, it's padding.
    vec = trained_embeddings[i] 
    tsv_writer.writerow(str(chr(i)))
#     out_m.write(chr(i+1), lineterminator='\n')# skip 0, it's padding.255 last vector
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

(127, 256)


Click me [Embeddings Projector](https://projector.tensorflow.org/) to visualize embeddings in 3D.

For ready pretrained models [TensorFlow Hub](https://tfhub.dev/)

I changed encoding scheme to cover UTF-8 encoded characters and the implemetation is found in the Scripts file.

## Transfer Learning

In [6]:
daily = tf.keras.models.load_model('daily.h5', compile=False)
daily.summary()

Model: "RNNStocks"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EmbedLayer (Embedding)       (None, None, 256)         32512     
_________________________________________________________________
BiLSTM (Bidirectional)       (None, 2048)              10493952  
_________________________________________________________________
BatchNormal (BatchNormalizat (None, 2048)              8192      
_________________________________________________________________
FullConnected (Dense)        (None, 512)               1049088   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
BatchNormal2 (BatchNormaliza (None, 512)               2048      
_________________________________________________________________
Output (Dense)               (None, 1)                 51

In [10]:
from tqdm import tqdm
import datetime
with pd.HDFStore('../news_db.h5', mode = 'r') as store:
    final_df = pd.DataFrame()
    for i in tqdm(store.keys()):
        df = store[i]
        yesterday = (datetime.datetime.now() + datetime.timedelta(-1)).strftime('%Y-%m-%d')
#        if df.loc[:, 'versionCreated'].apply(lambda x: x.strftime('%Y-%m-%d')) == yesterday:
        news_yesterday = df.loc[df.loc[:, 'versionCreated'].apply(lambda x: x.strftime('%Y-%m-%d')) == yesterday]
        news_yesterday = news_yesterday[~news_yesterday.text.duplicated()]
        if len(news_yesterday) > 0:
            sample = news_yesterday.text.str.cat(sep=' ')
            if len(sample) > 1000: sample = sample[:500] + "..." + sample[-500:]
            else: pass   
            temp_df = pd.DataFrame.from_dict({i.strip('/'): sample}, orient='index').rename(columns={0:'Headlines'})
            final_df = final_df.append(temp_df)
            final_df['Date'] = yesterday
            final_df = final_df[~final_df.Headlines.duplicated()]
        else: pass

100%|████████████████████████████████████████████████████████████████████████████████| 543/543 [05:08<00:00,  1.76it/s]


In [11]:
final_df['n_Characters'] = final_df['Headlines'].str.len()
final_df.describe().T

,count,mean,std,min,25%,50%,75%,max
n_Characters,292.0,361.304795,341.57085,20.0,93.75,215.5,550.75,1003.0


In [12]:
X = encode2bytes(final_df.Headlines.apply(lambda x: '<s>' + x + '<\s'))
X = pad_sequences(X, maxlen =  max(map(len, X)), padding = 'post', truncating='post')

predictions = daily.predict(X)

In [13]:
final_df['Predictions'] = np.squeeze(predictions)
final_df['Prediction Date'] = (datetime.datetime.now()).strftime('%Y-%m-%d')
final_df['BUY(20% Threshold)'] = (np.squeeze(predictions) > 0.2)
final_df['BUY(40% Threshold)'] = (np.squeeze(predictions) > 0.4)
final_df['BUY(60% Threshold)'] = (np.squeeze(predictions) > 0.6)

In [14]:
final_df[final_df['Headlines'].str.contains('NYSE ORDER IMBALANCE')][['Headlines', 'Predictions', 'Prediction Date']].to_csv('ORDER_IMBALANCES.csv')

Key Words:
*  blank check company
* SPAC
* 13F,G OR D
* Clinical Trials

In [24]:
# final_df[final_df['Headlines'].str.contains('13F')][['Headlines', 'Predictions', 'Prediction Date']].to_csv('key_words.csv')
final_df[final_df['Headlines'].str.contains('biotech')]

,Headlines,Date,n_Characters,Predictions,Prediction Date,BUY(20% Threshold),BUY(40% Threshold),BUY(60% Threshold)


In [55]:
final_df.to_csv('Todays_Prediction.csv')

In [252]:
sample = "Amazon's Sales Growth Costs a Fortune in Shipping and Fulfillment" + " Jeff Bezos, Bill Gates and other tech luminaries react to Biden's victory" + " Amazon rolls out rewards program that makes it easier for drivers to get work" + " TECH Alibaba cloud growth outpaces Amazon and Microsoft as Chinese tech giant pushes for profitability"
# sample = "Joe Biden" 
# sample = "Donald Trump" 
sample = '<s>' + sample + '<\s' 
print(sample)
sample = encode2bytes(sample)
print(sample)
sample = tf.ragged.constant(sample)
sample = tf.squeeze(sample, )
sample = tf.expand_dims(sample, 0).numpy()
print(sample)
sample.shape

In [179]:
predict = daily(sample).numpy()[0][0]
print("Probability from Headlines: %f" % predict)

Probability from Headlines: 0.099795


In [35]:
final_df[final_df.index == 'MRNA'].Headlines[0]

'BRAZIL\'S SUPREME COURT MAJORITY RULES TO ALLOW LOCAL GOVERNMENTS TO BYPASS FEDERAL GOVERNMENT TO BUY VACCINE IF IT FAILS TO DO SO NOVAVAX EXEC SAYS EXPECTS TO START PEDIATRIC STUDIES FOR ITS COVID-19 VACCINE IN SPRING - HOUSE SUBCOMMITTEE HEARING EXECS OF MODERNA, PFIZER, J&J, NOVAVAX AND ASTRAZENECA SAY THEY DO NOT CURRENTLY FORESEE ANY RAW MATERIAL SHORTAGES FOR MAKING THEIR COVID-19 VACCINES IN U.S. - HOUSE SUBCOMMITTEE HEARING EXCLUSIVE-AstraZeneca to miss second-quarter EU vaccine supply ta...O SOON RELAX SOME COVID GUIDELINES FOR PEOPLE WHO\'VE BEEN VACCINATED -CNN INTERVIEW Israel giving "symbolic" amounts of COVID vaccines to Honduras, others CORRECTED-UPDATE 4-More German state workers to get AstraZeneca jab as doses go begging CANADA SET TO RECEIVE RECORD 640,000 VACCINE DOSES THIS WEEK UPDATE 2-Pfizer to ship 13 mln COVID-19 vaccine doses per week to U.S. by mid-March, says executive Common side effects of Moderna and Pfizer-BioNTech COVID vaccines, ways to deal with them:

### Distribute Computations on Devices:

In [731]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]

In [737]:
with tf.device('/CPU'):
    xs = np.zeros((len(uniqye_characters),1))
    h_prev = np.zeros((10,1))
    Wxh = np.random.randn(10, len(uniqye_characters))*0.01 # input to hidden
    Whh = np.random.randn(10, 10)*0.01 # hidden to hidden
    Why = np.random.randn(len(uniqye_characters), 10)*0.01 # hidden to output
    bh = np.zeros((10, 1)) # hidden bias
    by = np.zeros((len(uniqye_characters), 1)) # output bias
    hs = np.tanh(np.dot(Wxh, xs) + np.dot(Whh, h_prev) + bh) # hidden state

In [41]:
!conda env export > environment.yml